<h1>Silly Google Ranking Algorithm</h1>

In [9]:
import numpy as np
import urllib.request
import os
import time
from tqdm.notebook import tqdm

In [5]:
#dictionary of urls
url_targets = {}
url_pointers = {}
#damping factor
d = 0.85
#creeper iterations
creeper_iterations=2
#base url
base_url = "https://en.wikipedia.org"
#initial url
initial_url = "/wiki/Black_Hole_Initiative"

<h1>Crawler</h1>

In [6]:
def crawler(initial_url, creeper_iterations):
    open_urls = [initial_url]
    url_pointers[initial_url] = 0
    url_targets[initial_url] = set()
    htmls = os.listdir('wiki')
    for n in range(creeper_iterations):
        print(n)
        new_urls = []
        for url in tqdm(open_urls):
            try:
                file_name = '_'.join(url.split('/')[2:])+'.html'
                if file_name in htmls:
                    file = open('wiki/'+file_name,encoding="utf-8").read()
                else:    
                    file = urllib.request.urlopen(base_url+url).read().decode("utf-8")
                    html = open('wiki/'+file_name,"w+",encoding="utf-8")
                    html.write(file)
                    html.close()
                file = file.replace(" ","")
                url_list =file.split('href="')
                for k in range(len(url_list)):
                    url_list[k] = url_list[k].split('"')[0]
                for found_url in url_list:
                    if "/wiki/" == found_url[:6] and not "#" in found_url and not ":" in found_url:
                        pointers = url_pointers.get(found_url,0)
                        if pointers == 0:
                            new_urls.append(found_url)
                        if not found_url in url_targets[url]:
                            url_targets[url].add(found_url)
                        url_pointers[url] += 1
                        url_pointers[found_url] = 0
                        url_targets[found_url] = set()
            except:
                print(file_name)
        open_urls = new_urls
    return url_pointers, url_targets

url_pointers, url_targets = crawler(initial_url, creeper_iterations)

0



1


<h1>Relaxation Method</h1>

We assume page $A$ has pages $T_1 ... T_n$ which point to it (i.e., are citations). The parameter $d$ is a damping factor which can be set between 0 and 1. We usually set $d$ to 0.85. There are more details about $d$ in the next section. Also $C(A)$ is defined as the number of links going out of page $A$. The PageRank of a page $A$ is given as follows:<br><br>
$$PR(A) = (1-d) + d (PR(T_1)/C(T_1) + ... + PR(T_n)/C(T_n))$$<br>
Note that the PageRanks form a probability distribution over web pages, so the sum of all web pages' PageRanks will be one.

In [12]:
start_time = time.time()

page_rank = {}

for key in url_pointers.keys():
    page_rank[key] = 1/len(url_pointers.keys())

for n in range(5):
    new_rank = {}
    for key in tqdm(page_rank.keys()):
        new_rank[key] = 1-d
        for sum_key in page_rank.keys():
            if key in url_targets[sum_key]:
                new_rank[key] += d*page_rank[sum_key]/len(url_targets[sum_key])
    page_rank = new_rank.copy()

page_rank = {url: rank for url, rank in sorted(page_rank.items(), key=lambda item: item[1], reverse=True)}

print('time to run: ',time.time()-start_time)


time to run:  378.9962568283081


In [13]:
list(page_rank)[:10]

['/wiki/Main_Page',
 '/wiki/Black_Hole_Initiative',
 '/wiki/Physical_cosmology',
 '/wiki/Big_Bang',
 '/wiki/Universe',
 '/wiki/Age_of_the_universe',
 '/wiki/Chronology_of_the_universe',
 '/wiki/Planck_epoch',
 '/wiki/Grand_unification_epoch',
 '/wiki/Electroweak_epoch']

In [14]:
[(key,page_rank[key]) for key in page_rank.keys()][:10]

[('/wiki/Main_Page', 0.2775),
 ('/wiki/Black_Hole_Initiative', 0.15000000000000002),
 ('/wiki/Physical_cosmology', 0.15000000000000002),
 ('/wiki/Big_Bang', 0.15000000000000002),
 ('/wiki/Universe', 0.15000000000000002),
 ('/wiki/Age_of_the_universe', 0.15000000000000002),
 ('/wiki/Chronology_of_the_universe', 0.15000000000000002),
 ('/wiki/Planck_epoch', 0.15000000000000002),
 ('/wiki/Grand_unification_epoch', 0.15000000000000002),
 ('/wiki/Electroweak_epoch', 0.15000000000000002)]

<h1>Linear Algebra</h1>

$$PR(A) = (1-d) + d (PR(T_1)/C(T_1) + ... + PR(T_n)/C(T_n))$$<br>
$$PR(A) - d (PR(T_1)/C(T_1) + ... + PR(T_n)/C(T_n)) = 1-d$$<br>
$$\begin{bmatrix}1 & -d/C_1(T_2) & ... \\ 
-d/C_2(T_1) & 1 & ... \\ \vdots & \vdots & \ddots \end{bmatrix} \begin{bmatrix}PR(A_1) \\ PR(A_2) \\ \vdots \end{bmatrix} = 
\begin{bmatrix}1-d \\ 1-d \\ \vdots \end{bmatrix}$$<br>
$$\begin{bmatrix}PR(A_1) \\ PR(A_2) \\ \vdots \end{bmatrix} = 
\begin{bmatrix}1-d \\ 1-d \\ \vdots \end{bmatrix}\begin{bmatrix}1 & -d/C_1(T_2) & ... \\ 
-d/C_2(T_1) & 1 & ... \\ \vdots & \vdots & \ddots \end{bmatrix}^{-1}$$

In [16]:
start_time = time.time()

keys = list(page_rank.keys())
b = np.zeros(len(keys))+1-d
A = np.identity(len(keys))
for row in tqdm(range(len(keys))):
    for column in range(len(keys)):
        if row != column and keys[column] in url_targets[keys[row]]:
            A[row,column] = -d/len(url_targets[keys[row]])
PR = np.matmul(b.T,np.linalg.inv(A))

print('time to run: ',time.time()-start_time)


time to run:  265.0543804168701


In [17]:
[(keys[n],PR[n]) for n in range(len(PR))][:10]

[('/wiki/Main_Page', 0.2775),
 ('/wiki/Black_Hole_Initiative', 0.15000000000000002),
 ('/wiki/Physical_cosmology', 0.15000000000000002),
 ('/wiki/Big_Bang', 0.15000000000000002),
 ('/wiki/Universe', 0.15000000000000002),
 ('/wiki/Age_of_the_universe', 0.15000000000000002),
 ('/wiki/Chronology_of_the_universe', 0.15000000000000002),
 ('/wiki/Planck_epoch', 0.15000000000000002),
 ('/wiki/Grand_unification_epoch', 0.15000000000000002),
 ('/wiki/Electroweak_epoch', 0.15000000000000002)]

<h1>Random Walk</h1>